In [54]:
from azureml.core.workspace import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

# get existing workspace
ws = Workspace(
    subscription_id="b14cff6a-38c4-4aec-9335-b453f4d03339",
    resource_group="RG-AIEAP",
    workspace_name="EAP",
)

print(ws.name, ws.resource_group, ws.location, sep="\n")

EAP
RG-AIEAP
southeastasia


In [2]:
from azureml.core import Experiment

experiment_name = "delta-experiment"
exp = Experiment(workspace=ws, name=experiment_name)

In [37]:
os.listdir()

['.ipynb_checkpoints',
 '20190903080554-4415.wav',
 'azureml-models',
 'fannoise-predictor.ipynb',
 'fold1',
 'fold2',
 'fold3',
 'fold4',
 'fold5',
 'model.h5',
 'myenv.yml',
 'samples-1.0.69',
 'score.py',
 'twocategories',
 'wav-150bands-150frames-3channel']

In [38]:
from azureml.core.model import Model

"""
model = Model.register(model_path = "model.h5",
    model_name = "fannoise-predictor",
    description = "delta fannoise recognotion model",
    workspace = ws)
"""

'\nmodel = Model.register(model_path = "model.h5",\n    model_name = "fannoise-predictor",\n    description = "delta fannoise recognotion model",\n    workspace = ws)\n'

In [39]:
# displaying registered models
models = ws.models
for name, m in models.items():
    print("Name:", name,"\nVersion:", m.version, "\nDescription:", m.description)

Name: fannoise-predictor 
Version: 1 
Description: delta fannoise recognotion model


#### Local testing model

In [40]:
import librosa
import numpy as np
import glob
from scipy import signal

model = Model.get_model_path(model_name = 'fannoise-predictor', _workspace = ws)

sample_audio = "20190903080554-4415.wav"

n_bands = 150
n_frames = 150
sample_rate = 22050


def read_audio(audio_path, target_fs=None, duration=4):
    (audio, fs) = librosa.load(audio_path, sr=None, duration=duration)
    # if this is not a mono sounds file
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    if target_fs is not None and fs != target_fs:
        audio = librosa.resample(audio, orig_sr=fs, target_sr=target_fs)
        fs = target_fs
    return audio, fs

def pad_trunc_seq_rewrite(x, max_len):
    if x.shape[1] < max_len:
        pad_shape = (x.shape[0], max_len - x.shape[1])
        pad = np.ones(pad_shape) * np.log(1e-8)
        x_new = np.hstack((x, pad))
    # no pad necessary - truncate
    else:
        x_new = x[:, 0:max_len]
    return x_new

def extract_features(parent_dir, sub_dirs, bands, frames, file_ext="*.wav"):
    # 4 second clip with 50% window overlap with small offset to guarantee frames
    n_window = int(sample_rate * 4. / frames * 2) - 4 * 2
    # 50% overlap
    n_overlap = int(n_window / 2.)
    # Mel filter bank
    melW = librosa.filters.mel(sr=sample_rate, n_fft=n_window, n_mels=bands, fmin=0., fmax=8000.)
    # Hamming window
    ham_win = np.hamming(n_window)
    log_specgrams_list = []

    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            # print("processing", fn)
            sound_clip, fn_fs = read_audio(fn, target_fs=sample_rate)
            assert (int(fn_fs) == sample_rate)

            if sound_clip.shape[0] < n_window:
                print("File %s is shorter than window size - DISCARDING - look into making the window larger." % fn)
                continue

            # Skip corrupted wavs
            if sound_clip.shape[0] == 0:
                print("File %s is corrupted!" % fn)
                continue

            # Compute spectrogram
            [f, t, x] = signal.spectral.spectrogram(
                x=sound_clip,
                window=ham_win,
                nperseg=n_window,
                noverlap=n_overlap,
                detrend=False,
                return_onesided=True,
                mode='magnitude')
            x = np.dot(x.T, melW.T)
            x = np.log(x + 1e-8)
            x = x.astype(np.float32).T
            x = pad_trunc_seq_rewrite(x, frames)

            log_specgrams_list.append(x)

    log_specgrams = np.asarray(log_specgrams_list).reshape(len(log_specgrams_list), bands, frames, 1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis=3)
    features = np.concatenate((features, np.zeros(np.shape(log_specgrams))), axis=3)
    for i in range(len(features)):
        # first order difference, computed over 9-step window
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
        # for using 3 dimensional array to use ResNet and other frameworks
        features[i, :, :, 2] = librosa.feature.delta(features[i, :, :, 1])

    return np.array(features)  #, np.array(labels, dtype=np.int)


features = extract_features(".", sample_audio, bands=n_bands, frames=n_frames)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [41]:
import keras
keras.backend.clear_session()

cnn = keras.models.load_model(model)

In [42]:
y_prob = cnn.predict(features, verbose=0)
y_pred = np.argmax(y_prob, axis=-1)

defect_code = {2: "Pass", 3: "Noise", 4: "Rpm", 5: "Vibration"}

print(defect_code[int(y_pred)])

Pass


In [76]:
%%writefile score.py
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse
from azureml.core.model import Model
import numpy as np
import keras
import librosa
from scipy import signal
import os

n_bands = 150
n_frames = 150
sample_rate = 22050


def read_audio(audio_path, target_fs=None, duration=4):
    (audio, fs) = librosa.load(audio_path, sr=None, duration=duration)
    # if this is not a mono sounds file
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    if target_fs is not None and fs != target_fs:
        audio = librosa.resample(audio, orig_sr=fs, target_sr=target_fs)
        fs = target_fs
    return audio, fs

def pad_trunc_seq_rewrite(x, max_len):
    if x.shape[1] < max_len:
        pad_shape = (x.shape[0], max_len - x.shape[1])
        pad = np.ones(pad_shape) * np.log(1e-8)
        x_new = np.hstack((x, pad))
    # no pad necessary - truncate
    else:
        x_new = x[:, 0:max_len]
    return x_new

def extract_features(sample_audio, bands, frames, file_ext="*.wav"):
    # 4 second clip with 50% window overlap with small offset to guarantee frames
    n_window = int(sample_rate * 4. / frames * 2) - 4 * 2
    # 50% overlap
    n_overlap = int(n_window / 2.)
    # Mel filter bank
    melW = librosa.filters.mel(sr=sample_rate, n_fft=n_window, n_mels=bands, fmin=0., fmax=8000.)
    # Hamming window
    ham_win = np.hamming(n_window)
    log_specgrams_list = []


    sound_clip, fn_fs = read_audio(sample_audio, target_fs=sample_rate)
    assert (int(fn_fs) == sample_rate)

    if sound_clip.shape[0] < n_window:
        print("File %s is shorter than window size - DISCARDING - look into making the window larger." % fn)

    # Skip corrupted wavs
    if sound_clip.shape[0] == 0:
        print("File %s is corrupted!" % fn)

    # Compute spectrogram
    [f, t, x] = signal.spectral.spectrogram(
        x=sound_clip,
        window=ham_win,
        nperseg=n_window,
        noverlap=n_overlap,
        detrend=False,
        return_onesided=True,
        mode='magnitude')
    x = np.dot(x.T, melW.T)
    x = np.log(x + 1e-8)
    x = x.astype(np.float32).T
    x = pad_trunc_seq_rewrite(x, frames)

    log_specgrams_list.append(x)

    log_specgrams = np.asarray(log_specgrams_list).reshape(len(log_specgrams_list), bands, frames, 1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis=3)
    features = np.concatenate((features, np.zeros(np.shape(log_specgrams))), axis=3)
    for i in range(len(features)):
        # first order difference, computed over 9-step window
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
        # for using 3 dimensional array to use ResNet and other frameworks
        features[i, :, :, 2] = librosa.feature.delta(features[i, :, :, 1])

    return np.array(features)  #, np.array(labels, dtype=np.int)


def init():
    global cnn
    model_path = Model.get_model_path(model_name = 'fannoise-predictor')
    keras.backend.clear_session()
    cnn = keras.models.load_model(model_path)


@rawhttp
def run(request):
    print("Request: [{0}]".format(request))
    
    
    if request.method == 'POST':
        file = request.files['file']
#         file.save('temp.wav')
#         reqBody = request.get_data(False)
#         # reqBody to sample_audio
#         sample_audio = reqBody
        
        features = extract_features(file, bands=n_bands, frames=n_frames)
        y_prob = cnn.predict(features, verbose=0)
        y_pred = np.argmax(y_prob, axis=-1)
        defect_code = {2: "Pass", 3: "Noise", 4: "Rpm", 5: "Vibration"}
        # For a real-world solution, you would load the data from reqBody
        # and send it to the model. Then return the response.
#         os.remove('temp.wav')

        # For demonstration purposes, this example just returns the posted data as the response.
        return AMLResponse(defect_code[int(y_pred)], 200)
    else:
        return AMLResponse("bad request", 500)

Overwriting score.py


In [82]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scipy")
myenv.add_conda_package("keras")
myenv.add_conda_package("librosa")
myenv.add_conda_package("numpy")
myenv.add_conda_package("tensorflow")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

In [83]:
from azureml.core.model import InferenceConfig
# Use environment in InferenceConfig
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

AttributeError: 'CondaDependencies' object has no attribute 'python'

In [58]:
# Rick sample code
from azureml.core.image import ContainerImage
from azureml.core.webservice import Webservice, AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "WAV",  
                                                     "method": "keras"},
                                               description='Predict wav file')
image_config = ContainerImage.image_configuration(
    execution_script = "score.py",
    runtime = "python",
    conda_file = "myenv.yml"
)


In [77]:
# Create the image
image = Image.create(name='myimg', models=[x[1] for x in models.items()], image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)

Creating image
Running.................................................................................................
Succeeded
Image creation operation finished for image myimg:1, operation "Succeeded"


In [84]:
# deploy from image
from azureml.core.image import Image
image = Image(ws, name ='myimg')
service = Webservice.deploy_from_image(
    name = "wavprediction",
    deployment_config = aci_config,
    image = image,
    workspace = ws
)

service.wait_for_deployment(show_output=True)

Running....................................
SucceededACI service creation operation finished, operation "Succeeded"


In [85]:
# debug
print(service.get_logs())

2019-10-31T08:02:05,723940013+00:00 - iot-server/run 
2019-10-31T08:02:05,725245716+00:00 - rsyslog/run 
2019-10-31T08:02:05,724776215+00:00 - gunicorn/run 
2019-10-31T08:02:05,869791169+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-10-31T08:02:09,550673338+00:00 - iot-server/finish 1 0
2019-10-31T08:02:09,553460344+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 45
generated new fontManager
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
2019-10-31 08:02:21,704 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerExce

In [ ]:
# Update score.py to service
service.update(inference_config = InferenceConfig(runtime="python",
                                   entry_script="score.py",
                                   conda_file="myenv.yml"))


In [86]:
# test service with sample
test_sample = bytes(test_sample, encoding='utf8')

prediction = service.run(input_data=test_sample)
print(prediction)

NameError: name 'test_sample' is not defined